In [1]:
# Puuttuvat kirjastot
!pip install tflite-model-maker
!pip install pycocotools

     |████████████████████████████████| 616 kB 5.7 MB/s 
     |████████████████████████████████| 120 kB 40.8 MB/s 
     |████████████████████████████████| 6.4 MB 20.9 MB/s 
     |████████████████████████████████| 1.2 MB 25.2 MB/s 
     |████████████████████████████████| 3.4 MB 29.7 MB/s 
     |████████████████████████████████| 840 kB 14.0 MB/s 
     |████████████████████████████████| 87 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 30.2 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 38.0 MB/s 
     |████████████████████████████████| 213 kB 55.7 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
     |████████████████████████████████| 25.3 MB 1.9 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 352 kB 37.3 MB/s 
     |████████████████████████████████| 47.6 MB 1.4 MB/s 
     |████████████████████████████████| 208 kB 38.0 MB/s 
  Created wheel for f

In [2]:
# Importit
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf

print(tf.__version__)

assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

from google.colab import files
from google.colab import drive
# file paths are returned so if you want to snag a list of downloaded files as you go, do this:

drive.mount("/content/drive") 

2.7.0
Mounted at /content/drive


In [3]:
# pohjamodel detection mallille
spec = model_spec.get('efficientdet_lite1')
# Muita malleja
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [4]:
# Määritetään train, test ja validation anotoidulle datalle, muulle kuin salaatti datalle se pitää itse luoda tensorflow2:n kustomoidulla csv-formaatilla
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

In [5]:
# Mallin koulutuksen käynnistys
# Batch size 4 tai 8, epoche 15-50
model = object_detector.create(train_data, model_spec=spec, batch_size=4, epochs=30, train_whole_model=True, validation_data=validation_data)

Epoch 1/30
43/43 [==============================] - 112s 1s/step - det_loss: 1.6956 - cls_loss: 1.1045 - box_loss: 0.0118 - reg_l2_loss: 0.0711 - loss: 1.7667 - learning_rate: 0.0065 - gradient_norm: 1.2397 - val_det_loss: 1.4696 - val_cls_loss: 0.9817 - val_box_loss: 0.0098 - val_reg_l2_loss: 0.0711 - val_loss: 1.5407
Epoch 2/30
43/43 [==============================] - 45s 1s/step - det_loss: 1.4090 - cls_loss: 0.9384 - box_loss: 0.0094 - reg_l2_loss: 0.0711 - loss: 1.4801 - learning_rate: 0.0050 - gradient_norm: 2.3286 - val_det_loss: 1.2294 - val_cls_loss: 0.7986 - val_box_loss: 0.0086 - val_reg_l2_loss: 0.0711 - val_loss: 1.3004
Epoch 3/30
43/43 [==============================] - 51s 1s/step - det_loss: 1.3019 - cls_loss: 0.8487 - box_loss: 0.0091 - reg_l2_loss: 0.0711 - loss: 1.3730 - learning_rate: 0.0049 - gradient_norm: 2.8310 - val_det_loss: 1.1545 - val_cls_loss: 0.7750 - val_box_loss: 0.0076 - val_reg_l2_loss: 0.0711 - val_loss: 1.2256
Epoch 4/30
43/43 [=====================

In [6]:
# Arvioidaan mallin luotettavuus/laatu
# Tavoite yli 0.5 mAP
model.evaluate(test_data)

1/1 [==============================] - 12s 12s/step



{'AP': 0.17042501,
 'AP50': 0.30653754,
 'AP75': 0.18887585,
 'AP_/Baked Goods': 0.0010592603,
 'AP_/Cheese': 0.13116509,
 'AP_/Salad': 0.45032609,
 'AP_/Seafood': 0.0016293377,
 'AP_/Tomato': 0.2679453,
 'APl': 0.17256297,
 'APm': 0.22183996,
 'APs': -1.0,
 'ARl': 0.32431313,
 'ARm': 0.425,
 'ARmax1': 0.12878565,
 'ARmax10': 0.2802674,
 'ARmax100': 0.32563502,
 'ARs': -1.0}

In [8]:
# Saved_modelin ja labels tiedoston tallennus
model.export(export_dir='/content/drive/MyDrive/', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])